### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
import joblib

### Load dataset

In [2]:
df = pd.read_csv("synthetic_mobile_money_transaction_dataset.csv")

### Data preprocessing

In [3]:
df.dropna(subset=['isFraud'], inplace=True)
df['transactionType'] = df['transactionType'].astype('category').cat.codes
df['initiator'] = df['initiator'].astype('category').cat.codes
df['recipient'] = df['recipient'].astype('category').cat.codes

for col in ['amount', 'oldBalRecipient', 'newBalRecipient']:
    df[col] = np.log1p(df[col])

features = ["step", "initiator", "recipient", "transactionType", "amount", "oldBalInitiator", "newBalInitiator", "oldBalRecipient", "newBalRecipient"]
target = "isFraud"

X = df[features]
y = df[target]

imputer = SimpleImputer(strategy='median')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

c:\Users\LAURENMAYA\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Initialize and train CatBoost model

In [4]:
catboost_model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0)
catboost_model.fit(X_train_scaled, y_train)

### Evaluate CatBoost model

In [5]:
y_pred = catboost_model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"CatBoost Evaluation:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

CatBoost Evaluation:
Accuracy: 0.9034
Precision: 0.7378
Recall: 0.0793
F1 Score: 0.1431


### Save the model

In [6]:
joblib.dump(catboost_model, "CatBoost.joblib")

['CatBoost.joblib']